In [1]:
import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v2
from datagenerator import ImageDataGenerator
from tensorflow.contrib.data import Iterator
import numpy as np
from datetime import datetime
import time
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2"

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Path to the textfiles for the trainings and validation set
num = 2
date = "20180808"
train_file = '/HS_code/0_Code/train%d_HR_ICROS.txt'%num
val_file1 = '/HS_code/0_Code/valid%d_HR_ICROS.txt'%num
batch_size = 128
num_classes = 2
learning_rate = 0.001
training_epochs = 50
display_step = 20
layer_num = 50

# Path for tf.summary.FileWriter and to store model checkpoints
filewriter_path = "/HS_code/2_Result_TB/tensorboard_HS_%s_ResNet_%d_hr%d_valid%d_%depoch_0_001"%(date,layer_num,num,num,training_epochs)
checkpoint_path = "/HS_code/1_Model_CP/checkpoints_HS_%s_ResNet_%d_hr%d_valid%d_%depoch_0_001"%(date,layer_num,num,num,training_epochs)

In [3]:
with tf.device('/gpu:2'):
    # data load
    tr_data = ImageDataGenerator(train_file,
                                 mode='training',
                                 batch_size=batch_size,
                                 num_classes=num_classes,
                                 shuffle=True)
    val_data1 = ImageDataGenerator(val_file1,
                                  mode='inference',
                                  batch_size=batch_size,
                                  num_classes=num_classes,
                                  shuffle=False)

     # create an reinitializable iterator given the dataset structure
    iterator = Iterator.from_structure(tr_data.data.output_types,
                                           tr_data.data.output_shapes)

    next_batch = iterator.get_next()
    

    # Ops for initializing the two different iterators
    training_init_op = iterator.make_initializer(tr_data.data)
    validation_init_op1 = iterator.make_initializer(val_data1.data)

    # TF placeholder for graph input and output
    x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
    y = tf.placeholder(tf.float32, [batch_size, num_classes])


    net, net_points = resnet_v2.resnet_v2_50(x, 
                                              num_classes=2,
                                              is_training=True)
        
    # Op for calculating the loss
    with tf.name_scope("cross_ent"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=y))
                
    # Train op
    with tf.name_scope("train"):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=y))
        train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
            
    # Add the loss to summary
    tf.summary.scalar('cross_entropy', loss)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(net[:,0,0], 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Add the accuracy to the summary
    tf.summary.scalar('accuracy', accuracy)
    
    # Merge all summaries together
    merged_summary = tf.summary.merge_all()

    # Initialize the FileWriter
    writer = tf.summary.FileWriter(filewriter_path)

    # Initialize an saver for store model checkpoints
    saver = tf.train.Saver()

    train_batches_per_epoch = int(np.floor(tr_data.data_size / batch_size))
    val_batches_per_epoch1 = int(np.floor(val_data1.data_size / batch_size)) 
    print(train_batches_per_epoch)

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.
Instructions for updating:
Replace `num_threads=T` with `num_parallel_calls=T`. Replace `output_buffer_size=N` with `ds.prefetch(N)` on the returned dataset.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
28


In [4]:
# Training
config=tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
config.gpu_options.allow_growth = True
with tf.Session(config=config, graph=tf.get_default_graph()) as sess:
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
   
    print("{} Start training...".format(datetime.now()))
    print("{} Open tensorboard --logdir={}".format(datetime.now(),
                                                      filewriter_path))
    
    img_batch = np.zeros((batch_size,227,227,3), dtype ='uint8')
    
    for epoch in range(training_epochs):
       
        print("{} Epoch number: {}".format(datetime.now(), epoch+1))
       
        # Initialize iterator with the training dataset
        sess.run(training_init_op)

        for step in range(train_batches_per_epoch):

            # get next batch of data
            img_batch, label_batch = sess.run(next_batch)       

            # And run the training op
            sess.run(train_op, feed_dict={x: img_batch, y: label_batch})

            # Generate summary with the current batch of data and write to file
            if step % display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: img_batch,
                                                        y: label_batch})
                writer.add_summary(s, epoch*train_batches_per_epoch + step)
                print("{} {} step".format(datetime.now(), step))

        # Validate the model on the entire validation set
        print("{} Start validation".format(datetime.now()))
        sess.run(validation_init_op1)
        test_acc = 0.
        test_count = 0
        
        for a in range(val_batches_per_epoch1):

            img_batch, label_batch = sess.run(next_batch)
            acc = sess.run(accuracy, feed_dict={x: img_batch,
                                                y: label_batch})
            test_acc += acc
            test_count += 1

        test_acc /= test_count
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                       test_acc))
        if epoch == training_epochs-1 :
            print("{} Saving checkpoint of model...".format(datetime.now()))
            # save checkpoint of the model
            checkpoint_name = os.path.join(checkpoint_path,
                                           'model_epoch'+str(epoch+1)+'.ckpt')
            save_path = saver.save(sess, checkpoint_name)

            print("{} Model checkpoint saved at {}".format(datetime.now(),
                                                           checkpoint_name))

2018-08-09 06:37:54.088434 Start training...
2018-08-09 06:37:54.089177 Open tensorboard --logdir=/HS_code/2_Result_TB/tensorboard_HS_20180808_ResNet_50_hr2_valid2_50epoch_0_001
2018-08-09 06:37:54.089594 Epoch number: 1
2018-08-09 06:37:59.254178 0 step
2018-08-09 06:38:13.301730 20 step
2018-08-09 06:38:17.984443 Start validation
2018-08-09 06:38:20.223636 Validation Accuracy = 0.7690
2018-08-09 06:38:20.224029 Epoch number: 2
2018-08-09 06:38:22.993053 0 step
2018-08-09 06:38:36.532028 20 step
2018-08-09 06:38:41.483422 Start validation
2018-08-09 06:38:43.867840 Validation Accuracy = 0.8136
2018-08-09 06:38:43.868014 Epoch number: 3
2018-08-09 06:38:46.668354 0 step
2018-08-09 06:39:00.251084 20 step
2018-08-09 06:39:05.016671 Start validation
2018-08-09 06:39:07.823661 Validation Accuracy = 0.8359
2018-08-09 06:39:07.823857 Epoch number: 4
2018-08-09 06:39:10.592893 0 step
2018-08-09 06:39:24.729727 20 step
2018-08-09 06:39:29.504280 Start validation
2018-08-09 06:39:32.370608 Val

2018-08-09 06:53:30.493314 Validation Accuracy = 0.9788
2018-08-09 06:53:30.493418 Epoch number: 39
2018-08-09 06:53:33.167735 0 step
2018-08-09 06:53:46.561256 20 step
2018-08-09 06:53:51.301701 Start validation
2018-08-09 06:53:54.080208 Validation Accuracy = 0.9710
2018-08-09 06:53:54.080394 Epoch number: 40
2018-08-09 06:53:56.771733 0 step
2018-08-09 06:54:10.342137 20 step
2018-08-09 06:54:15.079234 Start validation
2018-08-09 06:54:17.676459 Validation Accuracy = 0.9821
2018-08-09 06:54:17.676631 Epoch number: 41
2018-08-09 06:54:20.527557 0 step
2018-08-09 06:54:34.109135 20 step
2018-08-09 06:54:39.056958 Start validation
2018-08-09 06:54:41.318957 Validation Accuracy = 0.9810
2018-08-09 06:54:41.319146 Epoch number: 42
2018-08-09 06:54:44.037721 0 step
2018-08-09 06:54:57.618115 20 step
2018-08-09 06:55:02.526354 Start validation
2018-08-09 06:55:04.782846 Validation Accuracy = 0.9810
2018-08-09 06:55:04.782953 Epoch number: 43
2018-08-09 06:55:07.456611 0 step
2018-08-09 06: